<a href="https://colab.research.google.com/github/placerda/region-growing/blob/master/automatic_3d_region_growing_ijcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Import Libraries


In [0]:
!pip install pycuda pydicom
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

# if necessary include custom python modules use the except below
# import sys 
# sys.path.append('folder_name')

import numpy as np
import pydicom
import os
import pycuda.autoinit 
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import pandas as pd
import time
import cv2


In [0]:
# mount gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## Part 2: Region Growing Algorithm

CUDA C Region Growing Implementation.


In [0]:
ker = SourceModule("""

// seed detection parameters
#define _MIN_ORGAN_HU -700
#define _MAX_ORGAN_HU -600
#define _CHEBYSHEV_DIST 5

// region growing parameters
#define _NUM_FEATURES 5
#define _THRESHOLD 0.5
#define _DEPTH_WEIGHT 2.0

// depth feature parameters
#define _FIRST_SLICE 1
#define _MIN_DEPTH 1
#define _MAX_DEPTH 300

// image dimensions
#define _DEPTH  ( gridDim.z )
#define _WIDTH  512
#define _HEIGHT 512

// dicom HU range
#define _MIN_HU -1024
#define _MAX_HU 1024

// CUDA processing related constants
#define _X  ( threadIdx.x + blockIdx.x * blockDim.x )
#define _Y  ( threadIdx.y + blockIdx.y * blockDim.y )
#define _Z  ( threadIdx.z + blockIdx.z * blockDim.z )


__device__ int get_index(int x, int y, int z){
  // before calling this function test if coords are inside image boundaries
  return x  + y * _WIDTH + z * _WIDTH * _HEIGHT;
}

__device__ int get_neighbor_seed(int x, int y, int z, int *in)
// check if voxel is a region neighbor returning the neightbor's seed number
{
    int idx = get_index(x,y,z);
    for (int k = z-1; k <= z + 1; k++){
        for (int j = y-1; j <= y + 1; j++){
            for (int i = x-1; i <= x + 1; i++){
                // test if coords are inside image boundaries
                if (((k > 0) && (k < _DEPTH)) && ((j > 0) && (j < _HEIGHT)) && ((i > 0) && (i < _WIDTH))){
                    int idx_neighbor = get_index(i, j, k);
                    if (idx_neighbor != idx)
                        if (in[idx_neighbor] > 0) 
                            // if the neighbor is in the region 
                            // then returns its seed number
                            return in[idx_neighbor];
                }
            }
        }
    }  
    return 0;
}


__device__ float normalizeHU(int hu){
  // normalizes HU between _MIN_HU and _MAX_HU
  if (hu < _MIN_HU) 
    hu = _MIN_HU;
  else if (hu > _MAX_HU){
    hu = _MAX_HU;
  } 
  return ((float)abs(hu-_MIN_HU))/abs(_MAX_HU-_MIN_HU);
}


__device__ int calculate_features(int x, int y, int z, int *image, float *vector){  
  // calculates features vector (HU, MEAN, MIN, MAX)
  vector[0] = normalizeHU(image[get_index(x, y, z)]); 
  vector[1] = 0; // MEAN
  vector[2] = 0; // MIN
  vector[3] = 0; // MAX
  vector[4] = ((z * _DEPTH_WEIGHT * 1.0f) + _FIRST_SLICE)/ (_MAX_DEPTH - _MIN_DEPTH); // DEPTH

  float min  = 1;
  float max  = 0;
  float sum  = 0;
  float qtde = 0;
  
  // calculates: mean, min and max
  for (int k = z-1; k <= z+1; k++){
    for (int j = y-1; j <= y+1; j++){
      for (int i = x-1; i <= x+1; i++){
          if (((k > 0) && (k < _DEPTH)) && ((j > 0) && (j < _HEIGHT)) && ((i > 0) && (i < _WIDTH))){
            float hu = normalizeHU(image[get_index(i, j, k)]);
            sum+=hu;
            if (hu < min) min = hu;
            if (hu > max) max = hu;
            qtde++;
          }
      }
    }
  }

  vector[1] = (sum/qtde); // MEAN
  vector[2] = min; // MIN
  vector[3] = max; // MAX  
  return 0;
}

__device__ float euclidean_distance(float *vector1, float *vector2){
  float sum = 0;
  for (int i = 0; i < _NUM_FEATURES; i++){
    sum += pow((vector1[i] - vector2[i]), 2);
  }
  return (float)sqrt(sum);
}

__global__ void region_growing(int *image, int *region, int *seeds, int *elements_added, float *debug)
{

   int x = _X, y = _Y, z = _Z;
   if ((x < _WIDTH) && (y < _HEIGHT) && (z < _DEPTH)){
    int idx = get_index(x,y,z);
    if (region[idx] == 0) {  
         //element isn't inside the region
         int neighbor_seed = get_neighbor_seed(x, y, z, region);
         int neighbor_hu = image[get_index(x, y, z)];
         //if ((neighbor_seed > 0) && (neighbor_hu < 0)){
        if ((neighbor_seed > 0)){
             
            // get seed coordinates and calculates its feature vector
            int coord_offset = 3; 
            int idx_seed = neighbor_seed - 1;
            int seed_z = seeds[0+(idx_seed * coord_offset)];
            int seed_y = seeds[1+(idx_seed * coord_offset)];
            int seed_x = seeds[2+(idx_seed * coord_offset)];
            float vector1[_NUM_FEATURES];
            calculate_features(seed_x, seed_y, seed_z, image, vector1);

            // calculates element's feature vector
            float vector2[_NUM_FEATURES];
            calculate_features(x, y, z, image, vector2);

            // calculates distance
            float distance = euclidean_distance(vector1, vector2);

            // DEBUG
            //if ((x == 310) && (y == 231) && (z == 182)){
            //debug[0] =  (z  * 1.0f) / (_MAX_DEPTH - _FIRST_SLICE);
            //}
            // distance = 10;
            // DEBUG

            if (distance < _THRESHOLD){
                region[idx] = neighbor_seed;
                elements_added[0] += 1;
            }

         }
    }

   }

}

__global__ void find_seeds(int *image, int *seeds, float *debug)
{
    int chebyshev_dist = _CHEBYSHEV_DIST;
    int num_elements = (((1+chebyshev_dist)*chebyshev_dist)/2) * 8;
    int min_organ_hu = -700;
    int max_organ_hu = -550; //-600

    int x = _X, y = _Y, z = _Z;

    if ((x < _WIDTH) && (y < _HEIGHT) && (z < _DEPTH)){

        int sum = 0;
        int count = 0;
        int hu = image[get_index(x,y,z)];

        if ((hu > min_organ_hu) && (hu < max_organ_hu)){

            int min_hu = _MAX_HU ;
            int max_hu = _MIN_HU;
            
            // acummulates hu to calculate average
            for (int i=(x-chebyshev_dist); i<=(x+chebyshev_dist); i++)
                for (int j =(y-chebyshev_dist); j<=(y+chebyshev_dist); j++){
                    if ( ((j >= 0) && (j < _HEIGHT)) && 
                         ((i >= 0) && (i < _WIDTH)) &&
                        !((x == i) && (y == j))){
                            int idx = get_index(i,j,z);
                            sum += image[idx];
                            if (min_hu > image[idx]) min_hu = image[idx];
                            if (max_hu < image[idx]) max_hu = image[idx];
                            count++;
                        } 
                }
            
            if (count == num_elements){
                float average = (sum * 1.0f)/count;

                if ((average > min_organ_hu) && (average < max_organ_hu) && 
                    (min_hu > min_organ_hu) && (max_hu < max_organ_hu)){
                    // its a seed, add it to the seeds vector.
                    // its generating only one seed by now.
                    seeds[0] = z;
                    seeds[1] = y;
                    seeds[2] = x;
                }
            }

            //if ((z == 159) && (y == 287) && (x == 231)){
            //    debug[0] = z;
            //}

        }

    }
}
""")

## Part 3: CUDA Execution Configuration

Configure execution parameters.

This program uses the scans available in [scans](https://drive.google.com/drive/folders/1IzdrMYtHQFH176dkQszdXCGrIBLgb5dK).

Ground truth is available in [ground truth](https://drive.google.com/drive/folders/1PuGviaEh8i9Tbdac01P1RyeoZx-yB2yv).

In order to run this notebook in Google Colab you have to copy the dataset folder contents in your Google Drive path: **data/ratos/dataset**.


In [0]:
# program execution parameters

# dataset input and output folders
DICOM_FOLDER  = '/content/gdrive/My Drive/data/ratos/dataset'
RESULT_FOLDER = '/content/gdrive/My Drive/data/ratos/results'
SUMMARY_FOLDER = '/content/gdrive/My Drive/data/ratos/summary'

# filter scans to be segmented
SCANS = ['Imacx01_Animal1', 'Imacx01Animal2', 'Imacx2Animal2', 
         'Imacx2Animal3', 'Imacx2Animal5']

# dicom
FIRST_SLICE = 1   # z's axis ROI first slice
LAST_SLICE = 300  # z's axis ROI last slice
WIDTH = 512
HEIGHT = 512

# window configuration (256 grayscale quantization)
WINDOW_LENGHT=40
WINDOW_WIDTH=400

# processing flags
GENERATE_SUMMARY = True
GENERATE_IMAGES = False

## Part 4: Main Program

Process all scanes.


In [0]:
#  Summary Dataframe
summary_df = pd.DataFrame(columns=['Scan', 'Total', 'Hiper-aerated', 'Norm-aerated', 'Hypo-aerated', 'Non-aerated'])

# Process each Scan

for scan in SCANS:
    
    print ('\nscan: {}'.format(scan))

    # IMAGE ACQUISITION: loads the dicom image data into the main memory and copies it into the device's memory.

    dicom_folder = '{}/{}'.format(DICOM_FOLDER, scan)
    result_folder = '{}/{}'.format(RESULT_FOLDER, scan)

    files = os.listdir(dicom_folder)
    files.sort()

    last_slice = len(files) if len(files) < LAST_SLICE else LAST_SLICE
    num_slices = (last_slice - FIRST_SLICE) + 1
    
    h_image_data = np.zeros((num_slices, HEIGHT, WIDTH), dtype='int32')
    print ('loading slices {} to {}.'.format(FIRST_SLICE, last_slice))

    for idx, file in enumerate(files):
        # load dicom slices
        if (idx+1 >= FIRST_SLICE) and (idx < last_slice):
            filename = "{}/{}".format(dicom_folder, os.fsdecode(file))
            slice_index = idx + 1 - FIRST_SLICE

            ds = pydicom.dcmread(filename)
            b = ds.RescaleIntercept
            m = ds.RescaleSlope
            slice = m * ds.pixel_array + b
            slice = np.int32(slice)
            h_image_data[slice_index] = slice

            print('.', end='')

            if ((slice_index+1) % 100 == 0): print('') 
            
    d_image_data = gpuarray.to_gpu(h_image_data)

    # REGION GROWING: run the region growing loop until there are no more voxels to be included

    # initialize seed vector (only one seed in this version)
    h_seed_data = np.zeros((3), dtype='int32')
    d_seed_data = gpuarray.to_gpu(h_seed_data)

    find_seeds = ker.get_function("find_seeds")

    h_debug = np.zeros((1), dtype='float32')    
    d_debug = gpuarray.to_gpu(h_debug)

    find_seeds(d_image_data, d_seed_data, d_debug, 
            grid=(16, 16, num_slices), block=(32,32,1))

    h_seed_data = d_seed_data.get()
    if (h_seed_data.sum() > 0):
        print('found seed [z,y,x]: ', h_seed_data)
    else:
        print('could not find seed !!')        
        continue
    # initialize region data
    h_region_data = np.zeros((num_slices, HEIGHT, WIDTH), dtype='int32')

    # initialize region data with seeds, each element value is its seed number
    h_region_data[h_seed_data[0], h_seed_data[1], h_seed_data[2]] = 1
    d_region_data = gpuarray.to_gpu(h_region_data)

    region_growing = ker.get_function("region_growing")

    print('started region growing.')

    included = True
    loop_counter = 0
    while (included):
        h_elements_added = np.zeros((1), dtype='int32')
        d_elements_added = gpuarray.to_gpu(h_elements_added)
        h_debug = np.zeros((1), dtype='float32')    
        d_debug = gpuarray.to_gpu(h_debug)

        region_growing(d_image_data, d_region_data, d_seed_data, 
                       d_elements_added, d_debug, grid=(16, 16, num_slices), 
                       block=(32,32,1))
        pycuda.autoinit.context.synchronize()
        
        h_elements_added = d_elements_added.get()
        included = (h_elements_added[0] > 0)

        # used for debug only
        # h_debug = d_debug.get()    
        # print ('debug={}'.format(h_debug[0]))
            
        print('.', end='')
        loop_counter += 1
        if (loop_counter % 100 == 0): print('')

    h_segmented_data= d_region_data.get()
    h_segmented_data[h_segmented_data>0] = 1
    print('\nsegmented region volume: {}'.format(np.sum(h_segmented_data)))

    os.makedirs(result_folder, exist_ok=True)
    np.save('{}/segmented_data.npy'.format(result_folder), h_segmented_data)

    # SUMMARY: Generates segmentation summary

    if (GENERATE_SUMMARY):
        # Volume:
        # Hiper-aerated: -1024 a -900
        # Norm-aerated: -900 a -500
        # Hypo-aerated: -500 a -100
        # Non-aerated: -100 a +100
        a_row = {}
        a_row['Scan'] = scan

        # print('Total volume: ', h_segmented_data.sum())
        a_row['Total'] = h_segmented_data.sum()
        
        h_region = h_image_data.copy()
        h_region[h_segmented_data==0] = -4096

        h_sub_region = np.zeros_like(h_image_data)
        h_sub_region[(h_region>= -1024) & (h_region<-900)] = 1
        # print('Hiper-aerated: ', h_sub_region.sum())
        a_row['Hiper-aerated'] = h_sub_region.sum()

        h_sub_region = np.zeros_like(h_image_data)
        h_sub_region[(h_region>= -900) & (h_region<-500)] = 1
        # print('Norm-aerated: ', h_sub_region.sum())
        a_row['Norm-aerated'] = h_sub_region.sum()

        h_sub_region = np.zeros_like(h_image_data)
        h_sub_region[(h_region>= -500) & (h_region<-100)] = 1
        # print('Hypo-aerated: ', h_sub_region.sum())
        a_row['Hypo-aerated'] = h_sub_region.sum()

        h_sub_region = np.zeros_like(h_image_data)
        h_sub_region[(h_region>= -100) & (h_region<100)] = 1
        # print('Non-aerated: ', h_sub_region.sum())
        a_row['Non-aerated'] = h_sub_region.sum()

        summary_df = summary_df.append(a_row , ignore_index=True)

    # GENERATE IMAGES: Saves image in disk

    if (GENERATE_IMAGES):
        MIN_HU=-1024
        MAX_HU=1024

        print ('saving to disk slices {} to {}.'.format(FIRST_SLICE, last_slice))

        for idx, slice in enumerate(h_image_data):

            slice_number = str(idx+FIRST_SLICE).zfill(4)

            print ('.', end='')            
            # print ('{} '.format(slice_number), end='')
            if ((idx+1) % 100 == 0): print ('')
            
            # grayscale quantization
            min_value = WINDOW_LENGHT - (WINDOW_WIDTH // 2)
            if min_value < MIN_HU: min_value = MIN_HU
            max_value = WINDOW_LENGHT + (WINDOW_WIDTH // 2)
            if max_value > MAX_HU: max_value = MAX_HU
            quantized = slice.copy()
            quantized = np.clip(quantized, min_value, max_value)
            for cell in np.nditer(quantized, op_flags=['readwrite']):
                cell[...] = ((cell - min_value) * 255) // (max_value - min_value)
            gray = quantized.astype('uint8')

            # adds segmented data overlay
            mask = h_segmented_data[idx]
            alpha = 0.6
            masked_image = np.dstack((gray, gray, gray))
            overlay = masked_image.copy()
            overlay[mask > 0] =  (0,255,0)
            cv2.addWeighted(overlay, alpha, masked_image, 1 - alpha, 0, masked_image)

            region_area = np.sum(mask)

            # adds area and number info over image
            cv2.putText(masked_image, "region area={}".format(region_area), (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), lineType=cv2.LINE_AA)
            cv2.putText(masked_image, "{}".format(slice_number), (225, 30), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), lineType=cv2.LINE_AA)

            output_filepath = "{}/{}-auto.jpg".format(result_folder, slice_number)
            cv2.imwrite(output_filepath, masked_image)

# writes summary to disk
summary_df.to_csv('{}/summary-{}.csv'.format(SUMMARY_FOLDER, time.time()), index=False)
summary_df